In [102]:
import numpy as np
import cv2

map = cv2.imread("images\map2.png", 0)
map

array([[112, 148, 137, ..., 134, 133, 128],
       [139, 143, 110, ..., 130, 132, 118],
       [137, 131, 102, ..., 129, 133, 135],
       ...,
       [ 71,  72,  72, ..., 123, 123, 123],
       [ 73,  73,  73, ..., 123, 123, 123],
       [ 74,  74,  74, ..., 123, 123, 123]], dtype=uint8)

In [103]:
HEIGHT, WIDTH = map.shape

print(map)
rx, ry, rtheta = (WIDTH/4, HEIGHT/4, 0)

[[112 148 137 ... 134 133 128]
 [139 143 110 ... 130 132 118]
 [137 131 102 ... 129 133 135]
 ...
 [ 71  72  72 ... 123 123 123]
 [ 73  73  73 ... 123 123 123]
 [ 74  74  74 ... 123 123 123]]


In [104]:
STEP = 5
TURN = np.radians(25)

def get_input():
    fwd = 0
    turn = 0
    halt = False
    k = cv2.waitKey(0)
    if k == ord('w'):
        fwd = STEP
    elif k == ord('d'):
        turn = TURN
    elif k == ord('a'):
        turn = -TURN
    else:
        halt = True
    return fwd, turn, halt

In [105]:
MOVE_STEP = 0.5
MOVE_TURN = np.radians(5)
def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = np.random.normal(fwd, MOVE_STEP, 1)
    rx = rx + fwd_noisy*np.cos(rtheta)
    ry = ry + fwd_noisy*np.sin(rtheta)
    print("fwd noisy = ", fwd_noisy)
    
    turn_noisy = np.random.normal(turn, MOVE_TURN, 1)
    rtheta = rtheta + turn_noisy
    print("turn noisy = ", np.degrees(turn_noisy))
    
    return rx, ry, rtheta

In [106]:
NUM_PARTICLES = 3000

def init():
    particles = np.random.rand(NUM_PARTICLES, 3)
    particles *= np.array((WIDTH, HEIGHT, np.radians(360)) )
    return particles

In [107]:
def move_particles(particles, fwd, turn):
    particles[:,0] += fwd*np.cos(particles[:,2])
    particles[:,1] += fwd*np.sin(particles[:,2])
    particles[:,2] += turn
    
    particles[:,0] = np.clip(particles[:,0], 0.0, WIDTH-1)
    particles[:,1] = np.clip(particles[:,1], 0.0, HEIGHT-1)
    
    return particles

In [108]:
SIGMA_SENSOR = 2
def sense(x, y, noisy=False):
    SIGMA_SENSOR = 5
    x = int(x)
    y = int(y)
    if noisy:
        return map[y,x] + np.random.normal(0.0, SIGMA_SENSOR, 1)
    return map[y,x]

In [109]:
def compute_weights(particles, robot_sensor):    
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        particle_sensor = sense(particles[i,0], particles[i,1])
        errors[i] = abs(robot_sensor - particle_sensor)
    weights = np.max(errors) - errors
    
    weights[
        (particles[:,0] == 0) |
        (particles[:,0] == WIDTH-1) |
        (particles[:,1] == 0) |
        (particles[:,1] == HEIGHT-1)
    ] = 0.0

    weights = weights ** 3
    return weights

In [110]:
def resample(particles, weights):
    probabilities = weights / np.sum(weights)
    new_index = np.random.choice(
        NUM_PARTICLES,
        size=NUM_PARTICLES,
        p=probabilities
    )
    particles = particles[new_index,:]
    return particles

In [111]:
def add_noise(particles):
    SIGMA_PARTICLE_STEP = 2 #2
    SIGMA_PARTICLE_TURN = np.pi / 24 #np.pi / 24
    noise = np.concatenate((
        np.random.normal(0, SIGMA_PARTICLE_STEP, (NUM_PARTICLES,1)),
        np.random.normal(0, SIGMA_PARTICLE_STEP, (NUM_PARTICLES,1)),
        np.random.normal(0, SIGMA_PARTICLE_TURN, (NUM_PARTICLES,1)),
        ),
        axis=1
    )
    particles += noise
    return particles

In [112]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    
    # Display particles
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, 
                       (int(particles[i,0]), int(particles[i,1])), 
                       1, 
                       (255,0,0), 
                       1)
        
    # Display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)

    # Display best guess
    if len(particles) > 0:
        px = np.mean(particles[:,0])
        py = np.mean(particles[:,1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0,0,255), 5)

    cv2.imshow('map', lmap)

In [113]:
particles = init()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)        
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)
    
cv2.destroyAllWindows()                        


fwd noisy =  [4.94118439]
turn noisy =  [5.71114556]
fwd noisy =  [5.21660275]
turn noisy =  [3.14277275]
fwd noisy =  [4.82323984]
turn noisy =  [-7.53930694]
fwd noisy =  [5.54420179]
turn noisy =  [2.39904408]
fwd noisy =  [4.73973527]
turn noisy =  [-1.79463068]
fwd noisy =  [4.41336361]
turn noisy =  [-2.28978646]
fwd noisy =  [4.90206233]
turn noisy =  [-6.58668674]
fwd noisy =  [4.53384049]
turn noisy =  [9.02351969]
fwd noisy =  [4.38648481]
turn noisy =  [-0.56599383]
fwd noisy =  [6.03679449]
turn noisy =  [-7.70549269]
fwd noisy =  [5.03365639]
turn noisy =  [6.05650458]
fwd noisy =  [5.34634158]
turn noisy =  [5.04805235]
fwd noisy =  [4.86773833]
turn noisy =  [-2.51744822]
fwd noisy =  [4.7376237]
turn noisy =  [0.4932002]
fwd noisy =  [5.15897732]
turn noisy =  [0.7461778]
fwd noisy =  [5.38073087]
turn noisy =  [6.53171686]
fwd noisy =  [4.46938616]
turn noisy =  [-3.96587523]
fwd noisy =  [4.53663084]
turn noisy =  [-7.18220014]
fwd noisy =  [5.49014437]
turn noisy =  